# सामग्री तालिका
- [गाइडेंस के बारे में](../../../../code/01.Introduce)
- [सेटअप](../../../../code/01.Introduce)
- [असीमित जनरेशन](../../../../code/01.Introduce)
- [फाई 3 के लिए बोलना](../../../../code/01.Introduce)
- [रेगुलर एक्सप्रेशन](../../../../code/01.Introduce)
- [चयन](../../../../code/01.Introduce)
- [विचारों की श्रृंखला](../../../../code/01.Introduce)
- [JSON जनरेशन](../../../../code/01.Introduce)
- [HTML जनरेशन](../../../../code/01.Introduce)


# गाइडेंस के बारे में  
गाइडेंस एक सिद्ध ओपन-सोर्स Python लाइब्रेरी है जो किसी भी भाषा मॉडल (LM) के आउटपुट को नियंत्रित करने के लिए उपयोग की जाती है। केवल एक API कॉल के साथ, आप (Python में) सटीक प्रोग्रामेटिक बाधा(ओं) को व्यक्त कर सकते हैं, जिसे मॉडल को पालन करना होगा, और JSON, Python, HTML, SQL, या किसी भी संरचना में आउटपुट उत्पन्न कर सकते हैं जो उपयोग के मामले की आवश्यकता हो।  

गाइडेंस पारंपरिक प्रॉम्प्टिंग तकनीकों से अलग है। यह मॉडल को टोकन दर टोकन इंफेरेंस लेयर में निर्देशित करके बाधाओं को लागू करता है, जिससे उच्च गुणवत्ता वाले आउटपुट उत्पन्न होते हैं और अत्यधिक संरचित परिदृश्यों में उपयोग करते समय लागत और विलंबता को 30–50% तक कम करता है।  

गाइडेंस के बारे में अधिक जानने के लिए, [GitHub पर सार्वजनिक रिपॉजिटरी](https://github.com/guidance-ai/guidance) पर जाएं या Microsoft Build में [Guidance ब्रेकआउट सेशन](https://www.youtube.com/watch?v=qXMNPVVlCMs) देखें।  


# सेटअप
1. Guidance को `pip install guidance --pre` के साथ इंस्टॉल करें
2. Azure में Phi 3.5 mini endpoint को डिप्लॉय करें। इसके लिए https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml पर जाएं और "Deploy" बटन पर क्लिक करें
3. अपने endpoint की API key को `AZURE_PHI3_KEY` नामक एक environment variable में और URL को `AZURE_PHI3_URL` नामक एक environment variable में स्टोर करें


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# बिना किसी बाधा के निर्माण
पाठ को बिना किसी बाधा के `gen()` फ़ंक्शन का उपयोग करके उत्पन्न किया जा सकता है। यह Guidance के बिना मॉडल का उपयोग करने जैसा है।

## चैट स्वरूपण
कई चैट मॉडलों की तरह, Phi-3 उपयोगकर्ता और सहायक के बीच संदेशों को एक विशेष प्रारूप में अपेक्षित करता है। Guidance Phi-3 के चैट टेम्पलेट का समर्थन करता है और आपके लिए चैट स्वरूपण को प्रबंधित करेगा। चैट टर्न बनाने के लिए, बातचीत के प्रत्येक भाग को `with user()` या `with assistant()` ब्लॉक में रखें। सिस्टम संदेश सेट करने के लिए `with system()` ब्लॉक का उपयोग किया जा सकता है।


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## टोकन बचत
अत्यधिक संरचित परिस्थितियों में, Guidance टोकन को छोड़ सकता है और केवल आवश्यक टोकन उत्पन्न कर सकता है, जिससे प्रदर्शन में सुधार, दक्षता में वृद्धि और API लागत की बचत होती है। उत्पन्न टोकन इस नोटबुक में हाइलाइट किए गए बैकग्राउंड के साथ दिखाए गए हैं। मजबूर टोकन बिना हाइलाइटिंग के दिखाए जाते हैं और इनपुट टोकन के समान लागत रखते हैं, जो आउटपुट टोकन की लागत का लगभग एक तिहाई अनुमानित है।

*नोट:* बिना किसी प्रतिबंध के उत्पन्न करने वाले पहले उदाहरण में कोई टोकन मजबूर नहीं किया जा सका क्योंकि हमने कोई प्रतिबंध प्रदान नहीं किया।


# Phi 3 के लिए बोलना  
मार्गदर्शन के साथ, आप आसानी से मॉडल की प्रतिक्रियाओं में टेक्स्ट डाल सकते हैं। यह तब उपयोगी हो सकता है जब आप मॉडल के आउटपुट को किसी विशेष दिशा में ले जाना चाहते हैं।


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# रेगुलर एक्सप्रेशन के साथ सीमित करना  
पिछले उदाहरण में, Phi 3 ने `Canberra` के साथ सवाल का जवाब देने के बाद अतिरिक्त स्पष्टीकरण दिए। मॉडल के आउटपुट को केवल एक शब्द तक सीमित करने के लिए, एक रेगुलर एक्सप्रेशन का उपयोग किया जा सकता है।


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# कई विकल्पों में से चयन करना
जब कुछ संभावित विकल्प ज्ञात हों, तो आप मॉडल को विकल्पों की सूची में से चुनने के लिए `select()` फ़ंक्शन का उपयोग कर सकते हैं।


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` के साथ, केवल टोकन `Can` उत्पन्न हुआ। क्योंकि `Canberra` ही एकमात्र विकल्प है जो संभवतः उत्तर को पूरा कर सकता है, शेष टोकन मजबूर किए गए।


# विचारों की श्रृंखला
विचारों की श्रृंखला एक तकनीक है जो मॉडल के आउटपुट की गुणवत्ता को बेहतर बनाने में मदद कर सकती है, इसे समस्या को चरण दर चरण हल करने के लिए प्रोत्साहित करके। आमतौर पर, अंतिम उत्तर तक पहुँचने के लिए कई प्रॉम्प्ट टर्न की आवश्यकता होती है। पहले, मॉडल को चरण दर चरण सोचने के लिए निर्देशित करें। फिर, मॉडल को अंतिम उत्तर प्रदान करने के लिए पुनः प्रॉम्प्ट करें। मानक चैट इनफेरेंस APIs के साथ, इसमें 2 API कॉल्स लगते हैं, और मॉडल द्वारा उत्पन्न "विचारों की श्रृंखला" दो बार चार्ज होती है – एक बार आउटपुट टोकन के रूप में जब मॉडल इसे उत्पन्न करता है, और फिर दूसरी बार इनपुट टोकन के रूप में दूसरे कॉल के लिए। Guidance के साथ, पूरा बहु-चरणीय प्रक्रिया एक ही API कॉल के हिस्से के रूप में संसाधित और चार्ज की जाती है, जिससे लागत और विलंबता कम होती है।


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON निर्माण
Guidance का उपयोग JSON स्कीमा या pydantic मॉडल के साथ संगत JSON बनाने की गारंटी देने के लिए किया जा सकता है, जैसे कि यहां दिखाया गया उपयोगकर्ता प्रोफ़ाइल स्कीमा।


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML निर्माण

Guidance का उपयोग कोड बनाने और प्रोग्रामिंग भाषा में सिंटैक्स संबंधी आवश्यकताओं का पालन करने के लिए भी किया जा सकता है। इस अनुभाग में, हम बहुत ही सरल HTML वेबपेज लिखने के लिए एक छोटा Guidance प्रोग्राम बनाएंगे।

हम वेबपेज को छोटे-छोटे भागों में विभाजित करेंगे, जिनमें से प्रत्येक का अपना Guidance फ़ंक्शन होगा। फिर इन भागों को हमारे अंतिम फ़ंक्शन में मिलाकर एक HTML वेबपेज बनाया जाएगा।  
इसके बाद हम इस फ़ंक्शन को Azure AI में Guidance-सक्षम मॉडल के साथ चलाएंगे।

*नोट:* यह एक पूरी तरह से विकसित HTML जनरेटर नहीं होगा; इसका उद्देश्य यह दिखाना है कि आप अपनी व्यक्तिगत आवश्यकताओं के लिए संरचित आउटपुट कैसे बना सकते हैं।

हम Guidance से आवश्यक चीजें आयात करके शुरू करते हैं:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML वेबपेज अत्यधिक संरचित होते हैं, और हम Guidance का उपयोग करके पेज के उन हिस्सों को 'मजबूर' करेंगे।  
जब हमें मॉडल से स्पष्ट रूप से टेक्स्ट की आवश्यकता होती है, तो हमें यह सुनिश्चित करना होगा कि इसमें कोई भी ऐसा तत्व शामिल न हो जो टैग हो सकता है - यानी हमें '<' और '>' वर्णों को बाहर रखना होगा।


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

हम फिर इस फ़ंक्शन का उपयोग किसी भी मनमाने HTML टैग के भीतर टेक्स्ट उत्पन्न करने के लिए कर सकते हैं:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

अब, आइए पेज हेडर बनाते हैं।  
इसके तहत, हमें एक पेज शीर्षक तैयार करना होगा:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML पेज के मुख्य भाग को शीर्षकों और अनुच्छेदों से भरा जाएगा।  
हम प्रत्येक कार्य के लिए एक फ़ंक्शन परिभाषित कर सकते हैं:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

अब, HTML के मुख्य भाग को परिभाषित करने के लिए फ़ंक्शन।

यह `select()` का उपयोग करता है `recurse=True` के साथ, जिससे कई शीर्षक और पैराग्राफ उत्पन्न होते हैं:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

अब हम उस फ़ंक्शन पर आते हैं जो पूरी HTML पेज को जनरेट करता है।  
हम HTML की शुरुआत का टैग जोड़ते हैं, फिर हेडर जनरेट करते हैं, फिर बॉडी, और अंत में HTML का समाप्ति टैग जोड़ते हैं:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

हम एक उपयोगकर्ता-अनुकूल आवरण प्रदान करते हैं, जो हमें निम्नलिखित करने की अनुमति देगा:
- जेनरेशन का तापमान सेट करना
- Model ऑब्जेक्ट से उत्पन्न पृष्ठ को कैप्चर करना


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

हम फिर आउटपुट को एक फ़ाइल में लिख सकते हैं:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

और [परिणाम देखें](../../../../code/01.Introduce/sample_page.html)।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
